<a href="https://colab.research.google.com/github/mateollorente/Producto/blob/master/superVectorEvaluacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from google.colab import userdata
import os, json
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models


In [2]:
!git clone https://github.com/SantiagoBuffa/BeatAI.git
%cd BeatAI

Cloning into 'BeatAI'...
remote: Enumerating objects: 852, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 852 (delta 36), reused 43 (delta 13), pack-reused 762 (from 3)
Receiving objects: 100% (852/852), 98.16 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (566/566), done.
/content/BeatAI


In [15]:


kaggle_username = userdata.get("kaggle_username")
kaggle_key = userdata.get("kaggle_key")

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump({"username": kaggle_username, "key": kaggle_key}, f)

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

!pip install kaggle --quiet
!kaggle datasets download -d evilspirit05/ecg-analysis -p ./data --unzip

Dataset URL: https://www.kaggle.com/datasets/evilspirit05/ecg-analysis
License(s): MIT
100% 825M/826M [00:07<00:00, 42.5MB/s]
100% 826M/826M [00:07<00:00, 116MB/s] 


In [10]:
def apply_degradation_to_gray(gray_img):
    """
    Aplica un desenfoque Gaussiano y ruido normal a una imagen
    en escala de grises (array de NumPy).
    """
    # 1. Aplicar desenfoque (blur)
    blurred = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # 2. Añadir ruido
    # Convertimos a float32 para sumar el ruido sin problemas de clipping
    degraded_float = blurred.astype(np.float32)
    noise = np.random.normal(0, 10, gray_img.shape) # Ruido con std dev 10
    degraded_float += noise

    # 3. Volver al rango 0-255 y al tipo uint8
    degraded_clipped = np.clip(degraded_float, 0, 255)

    return degraded_clipped.astype(np.uint8)

In [11]:
def ecg_to_vector(path, num_rows=4, smooth=True, apply_degradation=False):
    """
    Tu función de extracción de señal, ahora con degradación opcional.
    """
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {path}")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    target_size = (1300, 930)
    gray = cv2.resize(gray, target_size, interpolation=cv2.INTER_AREA)
    # --- INICIO DE LA MODIFICACIÓN ---
    # Si se activa, aplicamos la degradación a la imagen en grises
    # ANTES de cualquier otro procesamiento.
    if apply_degradation:
        gray = apply_degradation_to_gray(gray)
    # --- FIN DE LA MODIFICACIÓN ---

    # El resto de tu lógica de binarización y limpieza funciona
    # ahora sobre la imagen 'gray' (limpia o degradada).
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY_INV, 35, 10)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 1))
    clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Manejar imágenes vacías (sin contornos)
    if len(contours) == 0:
        # Si no hay contornos, devuelve un vector de ceros
        # (o podrías decidir omitir este archivo en la función de carga)
        print(f"Advertencia: No se encontraron contornos en {path}. Saltando.")
        return None # Devolver None para que la función de carga lo omita

    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    clean = clean[y:y+h, x:x+w]

    height = clean.shape[0]

    # Evitar división por cero si la imagen es muy pequeña
    if height < num_rows:
        print(f"Advertencia: Altura de contorno ({height}) menor que num_rows en {path}. Saltando.")
        return None

    row_height = height // num_rows
    signals = []

    for i in range(num_rows):
        row = clean[i*row_height:(i+1)*row_height, :]
        ys = []
        for col in range(row.shape[1]):
            pixels = np.where(row[:, col] > 0)[0]
            if len(pixels) > 0:
                ys.append(np.mean(pixels))
            else:
                ys.append(np.nan)

        ys = np.array(ys)
        nans = np.isnan(ys)

        # Manejar caso donde todos son NaNs
        if np.all(nans) or not np.any(~nans):
            ys[:] = 0 # Rellenar con ceros si la tira está vacía
        elif np.any(nans):
            # Interpolar NaNs
            ys[nans] = np.interp(np.flatnonzero(nans), np.flatnonzero(~nans), ys[~nans])

        # Normalizar la señal de la tira (evitar división por cero)
        min_y, max_y = np.min(ys), np.max(ys)
        if max_y - min_y > 1e-6: # Un umbral pequeño
            ys = (ys - min_y) / (max_y - min_y)
        else:
            ys[:] = 0.5 # Si la línea es plana, ponerla en el medio

        signals.append(ys)

    vector = np.concatenate(signals)

    if smooth:
        vector = cv2.GaussianBlur(vector.reshape(-1, 1), (9, 1), 0).flatten()

    return vector

In [7]:
def resize_vector(v, target_len=4096):
    x_old = np.linspace(0, 1, len(v))
    x_new = np.linspace(0, 1, target_len)
    return np.interp(x_new, x_old, v)

In [14]:
def preprocess_dataset_1d(dataset_path, target_len=4096, num_rows=4):
    """
    Carga todas las imágenes del dataset y genera vectores 1D concatenados.
    - Aplica degradación al conjunto de 'train'.
    - NO aplica degradación al conjunto de 'test'.
    """
    train_dir = os.path.join(dataset_path, 'train')
    test_dir = os.path.join(dataset_path, 'test')

    X_train, y_train = [], []
    X_test, y_test = [], []

    class_names = sorted(os.listdir(train_dir))
    print(f"📂 Clases detectadas: {class_names}")

    # --- TRAIN (CON DEGRADACIÓN) ---
    for label, cls in enumerate(class_names):
        cls_dir = os.path.join(train_dir, cls)
        for fname in tqdm(os.listdir(cls_dir), desc=f"Procesando {cls} (train)"):
            path = os.path.join(cls_dir, fname)
            if not fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                continue

            # Llamamos con apply_degradation=True
            vec = ecg_to_vector(path, num_rows=num_rows, apply_degradation=True)

            # Omitir si la función devolvió None (por error o sin contornos)
            if vec is None:
                continue

            vec = resize_vector(vec, target_len)
            X_train.append(vec)
            y_train.append(label)

    # --- TEST (SIN DEGRADACIÓN) ---
    for label, cls in enumerate(class_names):
        cls_dir = os.path.join(test_dir, cls)
        for fname in tqdm(os.listdir(cls_dir), desc=f"Procesando {cls} (test)"):
            path = os.path.join(cls_dir, fname)
            if not fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                continue

            # Llamamos con apply_degradation=False
            vec = ecg_to_vector(path, num_rows=num_rows, apply_degradation=False)

            if vec is None:
                continue

            vec = resize_vector(vec, target_len)
            X_test.append(vec)
            y_test.append(label)

    X_train = np.array(X_train)[..., np.newaxis]
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(class_names))
    X_test = np.array(X_test)[..., np.newaxis]
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(class_names))

    # --- SPLIT VALIDATION ---
    # X_val se creará a partir de X_train, por lo que también
    # contendrá imágenes degradadas, lo cual es correcto.
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train.argmax(axis=1)
    )

    print(f"✅ X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")
    return X_train, y_train, X_val, y_val, X_test, y_test

In [12]:
def build_ecg_1d_model(input_length=4096, num_classes=4):
    """
    Construye el modelo CNN 1D.
    """
    model = models.Sequential([
        # El Input debe tener la forma (longitud, canales)
        layers.Input(shape=(input_length,)),
        # Reshape para añadir el canal (necesario para Conv1D)
        layers.Reshape((input_length, 1)),

        layers.Conv1D(32, 7, activation='relu', padding='same'),
        layers.MaxPooling1D(2),

        layers.Conv1D(64, 5, activation='relu', padding='same'),
        layers.MaxPooling1D(2),

        layers.Conv1D(128, 3, activation='relu', padding='same'),
        layers.GlobalAveragePooling1D(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
dataset_path = "/content/BeatAI/data/ECG_DATA"
TARGET_VECTOR_LENGTH = 4096
NUM_CLASSES = 4 # ¡Asegúrate de que esto coincida con tus datos!

# Llamamos a la nueva función de preprocesamiento
X_train, y_train, X_val, y_val, X_test, y_test = preprocess_dataset_1d(
    dataset_path
)

# Determinar el número de clases automáticamente
if y_train.shape[1] != NUM_CLASSES:
    print(f"Advertencia: NUM_CLASSES era {NUM_CLASSES}, pero los datos tienen {y_train.shape[1]} clases.")
    NUM_CLASSES = y_train.shape[1]

# Construir el modelo
model = build_ecg_1d_model(
    input_length=TARGET_VECTOR_LENGTH,
    num_classes=NUM_CLASSES
)

model.summary()

# Entrenar el modelo
if X_train.shape[0] > 0:
    print("\nIniciando entrenamiento...")
    history = model.fit(
        X_train, y_train,
        epochs=100, # Aumentado a 20 épocas
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )

    # Evaluar en el conjunto de test (limpio)
    print("\nEvaluando en el conjunto de test (limpio)...")
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc*100:.2f}%")
else:
    print("No se cargaron datos de entrenamiento. Revisa la ruta y los archivos.")

📂 Clases detectadas: ['ECG Images of Myocardial Infarction Patients (240x12=2880)', 'ECG Images of Patient that have History of MI (172x12=2064)', 'ECG Images of Patient that have abnormal heartbeat (233x12=2796)', 'Normal Person ECG Images (284x12=3408)']


Procesando ECG Images of Myocardial Infarction Patients (240x12=2880) (train): 100%|██████████| 956/956 [03:20<00:00,  4.76it/s]
Procesando ECG Images of Patient that have History of MI (172x12=2064) (train): 100%|██████████| 516/516 [01:43<00:00,  4.96it/s]
Procesando ECG Images of Patient that have abnormal heartbeat (233x12=2796) (train): 100%|██████████| 699/699 [02:19<00:00,  5.01it/s]
Procesando Normal Person ECG Images (284x12=3408) (train): 100%|██████████| 852/852 [02:50<00:00,  4.99it/s]
Procesando ECG Images of Myocardial Infarction Patients (240x12=2880) (test): 100%|██████████| 239/239 [00:31<00:00,  7.57it/s]
Procesando ECG Images of Patient that have History of MI (172x12=2064) (test): 100%|██████████| 172/172 [00:21<00:00,  8.03it/s]
Procesando ECG Images of Patient that have abnormal heartbeat (233x12=2796) (test): 100%|██████████| 233/233 [00:32<00:00,  7.09it/s]
Procesando Normal Person ECG Images (284x12=3408) (test): 100%|██████████| 284/284 [00:36<00:00,  7.84it/s

✅ X_train: (2418, 4096, 1), X_val: (605, 4096, 1), X_test: (928, 4096, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 4096, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4096, 32)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2048, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2048, 64)       │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1024, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1024, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,780 (171.02 KB)

 Trainable params: 43,780 (171.02 KB)

 Non-trainable params: 0 (0.00 B)


Iniciando entrenamiento...
Epoch 1/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 42s 511ms/step - accuracy: 0.2877 - loss: 1.3732 - val_accuracy: 0.3157 - val_loss: 1.3652
Epoch 2/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 31s 415ms/step - accuracy: 0.2846 - loss: 1.3655 - val_accuracy: 0.3157 - val_loss: 1.3580
Epoch 3/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 31s 403ms/step - accuracy: 0.3086 - loss: 1.3616 - val_accuracy: 0.3157 - val_loss: 1.3424
Epoch 4/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 40s 397ms/step - accuracy: 0.3279 - loss: 1.3356 - val_accuracy: 0.3240 - val_loss: 1.3380
Epoch 5/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 40s 387ms/step - accuracy: 0.3395 - loss: 1.3418 - val_accuracy: 0.3174 - val_loss: 1.3349
Epoch 6/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 29s 387ms/step - accuracy: 0.3323 - loss: 1.3343 - val_accuracy: 0.3107 - val_loss: 1.3402
Epoch 7/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 42s 398ms/step - accuracy: 0.3318 - loss: 1.3313 - val_accuracy: 0.3653 - val_loss: 1.3241
Epoch 8/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 31s 410ms/step - accuracy: 0.36

In [ ]:
def show_graphs(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

show_graphs(history)

In [ ]:
model.save("models/ecg_modelVectore.h5")

In [ ]:
#!git push origin main